In [1]:
import numpy as np

In [ ]:
list(zip({}, {}))

In [ ]:
def load_triggers(df, hg_threshold):
    triggers = []
    total_triggers = len(df) // 64

    for i in tqdm(range(total_triggers), desc="Processing triggers"):
        block = df.iloc[i*64:(i+1)*64]
        timestamp = block["Tstamp_us"].iloc[0]
        active_channels = block[block["HG"] > hg_threshold]["Ch"].values
        triggers.append({"timestamp": timestamp, "channels": active_channels})

    return triggers

In [ ]:
def calculate_efficiency_muon_tracks(triggers_D1, triggers_D2, X_D1, Y_D1, X_D2, Y_D2, match, tolerance, mode):
    if match:
        coincidences = match_triggers(triggers_D1, triggers_D2, tolerance)
    else:
        coincidences = list(zip(triggers_D1, triggers_D2))

    C_muon_i = {ch: 0 for ch in X_D1}
    C_muon_plus_i = {ch: 0 for ch in X_D1}
    C_muon_j = {ch: 0 for ch in Y_D1}
    C_muon_plus_j = {ch: 0 for ch in Y_D1}
    C_muon_i_D2 = {ch: 0 for ch in X_D2}
    C_muon_plus_i_D2 = {ch: 0 for ch in X_D2}
    C_muon_j_D2 = {ch: 0 for ch in Y_D2}
    C_muon_plus_j_D2 = {ch: 0 for ch in Y_D2}

    for pair in tqdm(coincidences, desc="Processing coincidences"):
        channels_D1 = set(pair[0]["channels"])
        channels_D2 = set(pair[1]["channels"])

        if mode == "vertical":
            for i in X_D1:
                for j in Y_D1:

                    # Channel i in D1
                    cond1 = (i - 1 in channels_D1 or i + 1 in channels_D1)
                    cond2 = (j in channels_D1 and (j - 1 in channels_D1 or j + 1 in channels_D1))
                    cond3 = (i in channels_D2 and (i - 1 in channels_D2 or i + 1 in channels_D2))
                    cond4 = (j in channels_D2 and (j - 1 in channels_D2 or j + 1 in channels_D2))

                    if cond1 and cond2 and cond3 and cond4:
                        if (i not in channels_D1):
                            C_muon_i[i] += 1
                            break
                        elif (i in channels_D1):
                            C_muon_plus_i[i] += 1
                            break

            for i in X_D2:
                for j in Y_D2:
                    # Channel j in D2
                    cond1 = (i - 1 in channels_D2 or i + 1 in channels_D2)
                    cond2 = (j in channels_D2 and (j - 1 in channels_D2 or j + 1 in channels_D2))
                    cond3 = (i in channels_D1 and (i - 1 in channels_D1 or i + 1 in channels_D1))
                    cond4 = (j in channels_D1 and (j - 1 in channels_D1 or j + 1 in channels_D1))

                    if cond1 and cond2 and cond3 and cond4:
                        if (i not in channels_D2):
                            C_muon_i_D2[i] += 1
                            break
                        elif (i in channels_D2):
                            C_muon_plus_i_D2[i] += 1
                            break

            for j in Y_D1:
                for i in X_D1:

                    # Channel i in D2
                    cond1 = (j - 1 in channels_D1 or j + 1 in channels_D1)
                    cond2 = (i in channels_D1 and (i - 1 in channels_D1 or i + 1 in channels_D1))
                    cond3 = (j in channels_D2 and (j - 1 in channels_D2 or j + 1 in channels_D2))
                    cond4 = (i in channels_D2 and (i - 1 in channels_D2 or i + 1 in channels_D2))

                    if cond1 and cond2 and cond3 and cond4:
                        if (j not in channels_D1):
                            C_muon_j[j] += 1
                            break
                        elif (j in channels_D1):
                            C_muon_plus_j[j] += 1
                            break

            for j in Y_D2:
                for i in X_D2:
                    # Channel j in D2
                    cond1 = (j - 1 in channels_D2 or j + 1 in channels_D2)
                    cond2 = (i in channels_D2 and (i - 1 in channels_D2 or i + 1 in channels_D2))
                    cond3 = (j in channels_D1 and (j - 1 in channels_D1 or j + 1 in channels_D1))
                    cond4 = (i in channels_D1 and (i - 1 in channels_D1 or i + 1 in channels_D1))

                    if cond1 and cond2 and cond3 and cond4:
                        if (j not in channels_D2):
                            C_muon_j_D2[j] += 1
                            break
                        elif (j in channels_D2):
                            C_muon_plus_j_D2[j] += 1
                            break

        elif mode == "all":

            for i in X_D1:
                vecinos_i = i - 1 in channels_D1 or i + 1 in channels_D1
                vecinos_j = any(j in channels_D1 and (j - 1 in channels_D1 or j + 1 in channels_D1) for j in Y_D1)
                vecinos_i_D2 = any(i2 in channels_D2 and (i2 - 1 in channels_D2 or i2 + 1 in channels_D2) for i2 in X_D2)
                vecinos_j_D2 = any(j2 in channels_D2 and (j2 - 1 in channels_D2 or j2 + 1 in channels_D2) for j2 in Y_D2)

                if vecinos_i and vecinos_j and vecinos_i_D2 and vecinos_j_D2:
                    if (i not in channels_D1):
                        C_muon_i[i] += 1
                    elif (i in channels_D1):
                        C_muon_plus_i[i] += 1

            for j in Y_D1:
                vecinos_j = j - 1 in channels_D1 or j + 1 in channels_D1
                vecinos_i = any(i in channels_D1 and (i - 1 in channels_D1 or i + 1 in channels_D1) for i in X_D1)
                vecinos_i_D2 = any(i2 in channels_D2 and (i2 - 1 in channels_D2 or i2 + 1 in channels_D2) for i2 in X_D2)
                vecinos_j_D2 = any(j2 in channels_D2 and (j2 - 1 in channels_D2 or j2 + 1 in channels_D2) for j2 in Y_D2)

                if vecinos_j and vecinos_i and vecinos_i_D2 and vecinos_j_D2:
                    if (j not in channels_D1):
                        C_muon_j[j] += 1
                    elif (j in channels_D1):
                        C_muon_plus_j[j] += 1
                        C_muon_j[j] += 1

            for i in X_D2:
                vecinos_i = i - 1 in channels_D2 or i + 1 in channels_D2
                vecinos_j = any(j in channels_D2 and (j - 1 in channels_D2 or j + 1 in channels_D2) for j in Y_D2)
                vecinos_i_D1 = any(i1 in channels_D1 and (i1 - 1 in channels_D1 or i1 + 1 in channels_D1) for i1 in X_D1)
                vecinos_j_D1 = any(j1 in channels_D1 and (j1 - 1 in channels_D1 or j1 + 1 in channels_D1) for j1 in Y_D1)

                if vecinos_i and vecinos_j and vecinos_i_D1 and vecinos_j_D1:
                    if (i not in channels_D2) and (len(channels_D2) == 3) and (len(channels_D1) == 4):
                        C_muon_i_D2[i] += 1
                    elif (i in channels_D2) and (len(channels_D2) == 4) and (len(channels_D1) == 4):
                        C_muon_plus_i_D2[i] += 1
                        C_muon_i_D2[i] += 1

            for j in Y_D2:
                vecinos_j = j - 1 in channels_D2 or j + 1 in channels_D2
                vecinos_i = any(i in channels_D2 and (i - 1 in channels_D2 or i + 1 in channels_D2) for i in X_D2)
                vecinos_i_D1 = any(i1 in channels_D1 and (i1 - 1 in channels_D1 or i1 + 1 in channels_D1) for i1 in X_D1)
                vecinos_j_D1 = any(j1 in channels_D1 and (j1 - 1 in channels_D1 or j1 + 1 in channels_D1) for j1 in Y_D1)

                if vecinos_j and vecinos_i and vecinos_i_D1 and vecinos_j_D1:
                    if (j not in channels_D2) and (len(channels_D2) == 3) and (len(channels_D1) == 4):
                        C_muon_j_D2[j] += 1
                    elif (j in channels_D2) and (len(channels_D2) == 4) and (len(channels_D1) == 4):
                        C_muon_plus_j_D2[j] += 1
                        C_muon_j_D2[j] += 1

        else:
            raise ValueError("The parameter 'mode' must be 'vertical' or 'all'.")

    efficiency_i, error_i = calc_eff_and_error(C_muon_i, C_muon_plus_i)
    efficiency_j, error_j = calc_eff_and_error(C_muon_j, C_muon_plus_j)
    efficiency_i_D2, error_i_D2 = calc_eff_and_error(C_muon_i_D2, C_muon_plus_i_D2)
    efficiency_j_D2, error_j_D2 = calc_eff_and_error(C_muon_j_D2, C_muon_plus_j_D2)

    effs = [efficiency_i, efficiency_j, efficiency_i_D2, efficiency_j_D2]
    C_muon = [C_muon_i, C_muon_j, C_muon_i_D2, C_muon_j_D2]
    C_muon_plus = [C_muon_plus_i, C_muon_plus_j, C_muon_plus_i_D2, C_muon_plus_j_D2]
    errors = [error_i, error_j, error_i_D2, error_j_D2]

    return effs, errors, C_muon, C_muon_plus